In [1]:
import shlex, subprocess

#Number of images 
N=input()

for i in range(1,N+1):
    
    #Generating audio files with festival
    command_line = 'text2wave Texto/0%d.txt'%i + ' -o Audio/0%d.wav'%i
    
    args = shlex.split(command_line)
    
    subprocess.Popen(args)     

3


In [2]:
#Duration
import subprocess
import re


f= open('Duration.txt','w') 
 
f.write('ffconcat version 1.0\n') 

for i in range(1,N+1):
    f.write('file  IMG/0%d.jpg \n'%i) 
    process = subprocess.Popen(['ffmpeg',  '-i','Audio/0%d.wav'%i], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    out,_ = process.communicate()
    result = re.search(r"Duration:\s{1}(?P<hours>\d+?):(?P<minutes>\d+?):(?P<seconds>\d+\.\d+?),", out, re.DOTALL).groupdict()
    f.write('duration  '+result['seconds']+'\n') 
f.close() 


In [3]:
#Generating video from images and archive duration

command_line ='ffmpeg -i Duration.txt  -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2"  Video0.mp4'
    
args = shlex.split(command_line)
    
subprocess.Popen(args) 

In [4]:
#Union of audios
f= open('Aud.txt','w') 
for i in range(1,N+1):
    f.write('file Audio/0%d.wav \n'%i)
f.close()

command_line ='ffmpeg -f concat -safe 0 -i Aud.txt -c copy Audio/Audio.wav'
    
args = shlex.split(command_line)
    
subprocess.Popen(args) 

In [7]:
#Concatenate Video with Audio

command_line ='ffmpeg -i Video0.mp4 -i Audio/Audio.wav  Video00.mp4'
    
args = shlex.split(command_line)
    
subprocess.Popen(args) 